# Lab 3

### Context
#### Cross Validation
+ The Set of Train, Valid, Test 
+ k-Fold Cross Validation with Stratify

#### Ensemble
+ Voting Ensemble
+ Out-of-fold(OOF) Ensemble

In [ ]:
import os
from os.path import join

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
BASE_DIR = ''

train_path = join(BASE_DIR, 'data', 'MDC14', 'train.csv')
test_path  = join(BASE_DIR, 'data', 'MDC14', 'test.csv')

data = pd.read_csv(train_path)
test = pd.read_csv(test_path)

label = data['credit']

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
test.head()

In [ ]:
test.describe()

In [ ]:
test.info()

In [ ]:
# 불필요한 컬럼 제거
data.drop(columns=['index', 'credit'], inplace=True)
test.drop(columns=['index'],           inplace=True)

In [ ]:
cat_columns = [c for c, t in zip(data.dtypes.index, data.dtypes) if t == 'O'] 
num_columns = [c for c    in data.columns if c not in cat_columns]

print('Categorical Columns: \n{}\n'.format(cat_columns))
print('Numeric Columns: \n{}'.format(num_columns))

#### 라벨 데이터 인코딩

In [ ]:
label = label.astype(int)

## Cross Validation
### 1. Train, Valid, Test Set
훈련, 검증, 테스트 데이터라고 부르는 3가지를 한번 이야기 해보겠습니다.<br>
* Train Data : 모델을 학습하는데 사용하는 데이터 (모델이 알고 있는 학습할 데이터, 과거 데이터)
* Valid Data : 학습한 모델의 성능을 검증하는 데이터 (모델이 모르는 학습하지 않을 데이터, 모델 검증에 사용하는 데이터, 과거 데이터)
* Test Data : 학습한 모델로 예측할 데이터 (모델이 모르는 예측할 데이터, 미래 데이터)

<img src='./img/train_val_test.png' style='height : 500px' >


##### 데이터 쪼개기, Train -> (Train, Valid)
- train_test_split 파라미터 
    - test_size  (float): Valid(test)의 크기의 비율을 지정
    - random_state (int): 데이터를 쪼갤 때 내부적으로 사용되는 난수 값 (해당 값을 지정하지 않으면 매번 달라집니다.)
    - shuffle     (bool): 데이터를 쪼갤 때 섞을지 유무
    - stratify   (array): Stratify란, 쪼개기 이전의 클래스 비율을 쪼개고 나서도 유지하기 위해 설정해야하는 값입니다. 클래스 라벨을 넣어주면 됩니다.
        - ex) 원본 Train 데이터의 클래스 비율이 (7:3) 이었다면, 쪼개어진 Train, Valid(test) 데이터의 클래스 비율도 (7:3)이 됩니다. 당연히 분류 데이터에서만 사용할 수 있습니다.
        
#### ref
- [검증 데이터가 필요한 이유 3months님 블로그](https://3months.tistory.com/118)

In [ ]:
from sklearn.model_selection import train_test_split

# 쪼개어진 Train, Valid 데이터의 비율은 (7:3), 내부 난수 값 42, 데이터를 쪼갤 때 섞으며 label 값으로 Stratify 하는 코드 입니다. random_state를 주석 처리하고 데이터를 확인해보시면 계속 바뀝니다.
x_train, x_valid, y_train, y_valid = train_test_split(data, label, 
                                                      test_size=0.3,
                                                      random_state=42,
                                                      shuffle=True,
                                                      stratify=label)

In [ ]:
data.shape, x_train.shape, x_valid.shape

### 2. k-fold with stratify
k-fold는 데이터를 k개로 쪼개는 것을 말합니다. <br>
일반적으로 Cross Validation에서 사용되며, 데이터셋을 k개로 쪼개어 k-1개로 모델을 학습하고, 1개로 모델을 검증합니다. <br>
k개로 데이터를 쪼개면, 모든 fold에 대해(하나의 fold를 선택하여) 검증하는 방식으로 k번 다른 데이터셋으로 학습한 모델을 검증할 수 있습니다.

![kfold](./img/kfold.png)

#### Stratify, 계층적 k-fold는 뭔가요?
k-fold는 데이터의 정렬 유무와 분류할 클래스의 비율에 상관없이 순서대로 데이터를 분할하는 특징이 있습니다.<br>
하지만, 분류할 클래스의 비율이 다르다면 어떻게 될까요? 그런 경우에는, 각 fold가 학습 데이터셋을 대표한다고 말하기 어려워집니다.<br>
한 fold에 특정 클래스가 많이 나올수도, 적게 나올수도 있기 때문입니다. Stratified k-fold는 그러한 문제점을 해결하기 위해 제안되었습니다.<br>
k개의 fold도 분할한 이후에도, 전체 훈련 데이터의 클래스 비율과 각 fold가 가지고 있는 클래스의 비율을 맞추어 준다는 점이 기존의 k-fold와의 다른 특징 입니다. 

##### k-fold
![kfold_example](./img/kfold_example.png)

##### Stratified k-fold
![stratified_kfold_example](./img/stratified_kfold_example.png)

#### k-Fold
k-fold는 말 그대로 데이터를 k개로 쪼갭니다. <br>
k의 개수를 조절하여 몇개의 fold를 만들지 결정할 수 있습니다.

k-fold는 sklearn의 model_selection 패키지에 있습니다.

- KFold 대표 파라미터
    - n_splits      (int)  : Fold의 개수 k 값
    - shuffle       (bool) : 데이터를 쪼갤 때 섞을지 유무
    - random_state  (int)  : 내부적으로 사용되는 난수값
    
#### ref
- [Scikit-learn, KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)
- [Scikit-learn, Stratified-KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold)
- [Scikit-learn, Compare with KFold, StratifedKFold](https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py)

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

In [ ]:
for i, (trn_idx, val_idx) in enumerate(kf.split(data, label)):
    x_train, y_train = data.iloc[trn_idx, :], label.iloc[trn_idx,]
    x_valid, y_valid = data.iloc[val_idx, :], label.iloc[val_idx,]
    
    print('{} Fold, trn label\n Class 0: {}, Class 1: {}, Class 2: {}'.format(i,   np.sum(y_train == 0), np.sum(y_train == 1), np.sum(y_train == 2)))
    print('{} Fold, val label\n Class 0: {}, Class 1: {}, Class 2: {}\n'.format(i, np.sum(y_valid == 0), np.sum(y_valid == 1), np.sum(y_valid == 2)))

#### stratify k-Fold

Stratified k-fold는 sklearn의 model_selection 패키지에 있습니다.

- StratifiedKFold 대표 파라미터
    - n_splits      (int)  : Fold의 개수 k 값
    - shuffle       (bool) : 데이터를 쪼갤 때 섞을지 유무
    - random_state  (int)  : 내부적으로 사용되는 난수값

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)

In [ ]:
for i, (trn_idx, val_idx) in enumerate(skf.split(data, label)):
    x_train, y_train = data.iloc[trn_idx, :], label.iloc[trn_idx,]
    x_valid, y_valid = data.iloc[val_idx, :], label.iloc[val_idx,]
    
    print('{} Fold, trn label\n Class 0: {}, Class 1: {}, Class 2: {}'.format(i,   np.sum(y_train == 0), np.sum(y_train == 1), np.sum(y_train == 2)))
    print('{} Fold, val label\n Class 0: {}, Class 1: {}, Class 2: {}\n'.format(i, np.sum(y_valid == 0), np.sum(y_valid == 1), np.sum(y_valid == 2)))

#### 전처리 프로세스 함수로 작성

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def preprocess(x_train, x_valid, x_test):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    tmp_x_test  = x_test.copy()
    
    
    # 결측치 처리
    
    
    # 스케일링
    

    # 인코딩
    
    return tmp_x_train, tmp_x_valid, tmp_x_test

#### Cross Validation 해보기 (실습 15분)
Stratified k-fold를 이용해 Cross Validation을 진행해 보겠습니다. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics  import log_loss

val_scores = list()

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i, (trn_idx, val_idx) in enumerate(skf.split(data, label)):
    x_train, y_train = data.iloc[trn_idx], label.iloc[trn_idx]
    x_valid, y_valid = data.iloc[val_idx], label.iloc[val_idx]
    
    # 전처리
    
    
    # 모델 정의
    
    
    # 모델 학습
    
    
    # 훈련, 검증 데이터 log_loss 확인
    trn_logloss = 
    val_logloss = 
    print('{} Fold, train logloss : {:.4f}4, validation logloss : {:.4f}'.format(i, trn_logloss, val_logloss))
    
    val_scores.append(val_logloss)

# 교차 검증 logloss 평균 계산하기
print('Cross Validation Score : {:.4f}'.format(np.mean(val_scores)))

## Ensemble
개인적으로 앙상블은 머신러닝의 꽃이라고 생각합니다. 단일 모델로 좋은 성능을 이끄는 것도 중요하지만, 서로 다른 모델의 다양성을 고려하여 결과를 이끌어내는 앙상블은 응용할 수 있는 방법이 매우 많습니다. <br>
그 중 대표적인 2가지 앙상블에 대해 실습하고 배워보도록 하겠습니다. 

### 1. Voting Ensemble
이름에서 알 수 있듯이 각자의 모델이 투표를 하여 클래스를 선택하는 방식의 앙상블 입니다. <br>
Voting 앙상블은 Sklearn 자체적으로 모델로써 지원을 하며, 사용하기도 매우 쉽습니다. <br>
그리고 Hard, Soft로 Voting 방식이 나뉘는데, Hard는 라벨 값으로 투표를 하는 방식이고, Soft는 확률 값을 모두 더해 가장 높은 클래스를 선택합니다.

Voting Classifier는 Sklearn의 ensemble 패키지에 있습니다.

#### 1) 모델 불러오기 및 정의하기

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

clfs = [['Logistic', LogisticRegression()],
        ['RandomForest', RandomForestClassifier()],
        ['MLP', MLPClassifier()]]

vote_clf = VotingClassifier(clfs, voting='soft', n_jobs=4)

#### 2) 모델 학습하기

In [ ]:
# 여기서 x_train, y_train은 마지막 Fold
vote_clf.fit(x_train, y_train)

#### 3) 결과 확인하기

In [ ]:
print('Validation logloss : {:.4f}'.format(log_loss(y_valid, vote_clf.predict_proba(x_valid))))

### 2. Out-of-fold(OOF) Ensemble (같이 푸는 실습)
OOF 앙상블은 KFold 교차 검증에서 생성되는 각 Fold에 대한 예측 값을 앙상블하는 기법으로 모델 검증과 함께 앙상블을 진행할 수 있다는 장점이 있습니다. <br>

Cross Validation 파트에서 배웠던 KFold 코드를 재사용해 OOF 앙상블을 진행해보겠습니다.

In [ ]:
val_scores = list()
oof_pred = np.zeros((#맞는 차원 집어넣기))

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i, (trn_idx, val_idx) in enumerate(skf.split(data, label)):
    x_train, y_train = data.iloc[trn_idx, :], label.iloc[trn_idx,]
    x_valid, y_valid = data.iloc[val_idx, :], label.iloc[val_idx,]
    
    # 전처리
    
    
    # 모델 정의
    
    
    # 모델 학습

    
    # 훈련, 검증 데이터 log_loss 확인
    trn_logloss = 
    val_logloss = 
    print('{} Fold, train logloss : {:.4f}4, validation logloss : {:.4f}'.format(i, trn_logloss, val_logloss))
    
    val_scores.append(val_logloss)
    
    oof_pred += clf.predict_proba(x_test) / skf.n_splits 

# 교차 검증 정확도 평균 계산하기
print('Cross Validation Score : {:.4f}'.format(np.mean(val_scores)))

In [ ]:
submit_path = join(BASE_DIR, 'data', 'MDC14', 'sample_submission.csv')

submit = pd.read_csv(submit_path)
submit

In [ ]:
submit.iloc[:, 1:] = oof_pred
submit

In [ ]:
# submit.to_csv('oof_first_submit.csv', index=False)

## 실습 솔루션

### 1) 전처리 함수 만들기

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def preprocess(x_train, x_valid, x_test):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    tmp_x_test  = x_test.copy()
    
    tmp_x_train.reset_index(drop=True, inplace=True)
    tmp_x_valid.reset_index(drop=True, inplace=True)
    
    # 결측치 처리
    imputer = SimpleImputer(strategy='most_frequent')
    tmp_x_train[cat_columns] = imputer.fit_transform(tmp_x_train[cat_columns])
    tmp_x_valid[cat_columns] = imputer.transform(tmp_x_valid[cat_columns])
    tmp_x_test[cat_columns]  = imputer.transform(tmp_x_test[cat_columns])
    
    # 스케일링
    scaler = StandardScaler()
    tmp_x_train[num_columns] = scaler.fit_transform(tmp_x_train[num_columns])
    tmp_x_valid[num_columns] = scaler.transform(tmp_x_valid[num_columns])
    tmp_x_test[num_columns]  = scaler.transform(tmp_x_test[num_columns])

    # 인코딩
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(tmp_x_train[cat_columns])
    
    tmp_x_train_cat = pd.DataFrame(ohe.transform(tmp_x_train[cat_columns]))
    tmp_x_valid_cat = pd.DataFrame(ohe.transform(tmp_x_valid[cat_columns]))
    tmp_x_test_cat  = pd.DataFrame(ohe.transform(tmp_x_test[cat_columns]))
    
    tmp_x_train.drop(columns=cat_columns, inplace=True)
    tmp_x_valid.drop(columns=cat_columns, inplace=True)
    tmp_x_test.drop(columns=cat_columns, inplace=True)
    
    tmp_x_train = pd.concat([tmp_x_train, tmp_x_train_cat], axis=1)
    tmp_x_valid = pd.concat([tmp_x_valid, tmp_x_valid_cat], axis=1)
    tmp_x_test  = pd.concat([tmp_x_test, tmp_x_test_cat], axis=1)
    
    return tmp_x_train, tmp_x_valid, tmp_x_test

### 2) Cross Validation  실습 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics  import log_loss

val_scores = list()

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i, (trn_idx, val_idx) in enumerate(skf.split(data, label)):
    x_train, y_train = data.iloc[trn_idx], label.iloc[trn_idx]
    x_valid, y_valid = data.iloc[val_idx], label.iloc[val_idx]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)
    
    # 모델 정의
    model = RandomForestClassifier(n_estimators=100, n_jobs=-1)
    
    # 모델 학습
    model.fit(x_train, y_train)
    
    # 훈련, 검증 데이터 log_loss 확인
    trn_logloss = log_loss(y_train, model.predict_proba(x_train))
    val_logloss = log_loss(y_valid, model.predict_proba(x_valid))
    print('{} Fold, train logloss : {:.4f}4, validation logloss : {:.4f}'.format(i, trn_logloss, val_logloss))
    
    val_scores.append(val_logloss)

# 교차 검증 logloss 평균 계산하기
print('Cross Validation Score : {:.4f}'.format(np.mean(val_scores)))

In [ ]:
val_scores = list()
oof_pred = np.zeros((test.shape[0], 3))

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i, (trn_idx, val_idx) in enumerate(skf.split(data, label)):
    x_train, y_train = data.iloc[trn_idx, :], label.iloc[trn_idx,]
    x_valid, y_valid = data.iloc[val_idx, :], label.iloc[val_idx,]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)
    
    # 모델 정의
    clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    
    # 모델 학습
    clf.fit(x_train, y_train)

    # 훈련, 검증 데이터 log_loss 확인
    trn_logloss = log_loss(y_train, model.predict_proba(x_train))
    val_logloss = log_loss(y_valid, model.predict_proba(x_valid))
    print('{} Fold, train logloss : {:.4f}4, validation logloss : {:.4f}'.format(i, trn_logloss, val_logloss))
    
    val_scores.append(val_logloss)
    
    oof_pred += clf.predict_proba(x_test) / skf.n_splits 

# 교차 검증 정확도 평균 계산하기
print('Cross Validation Score : {:.4f}'.format(np.mean(val_scores)))